# Feature Engineering - House Rocket Insights Project

## 0.0. IMPORTS

In [1]:
import math
import pandas  as pd
import numpy   as np
import seaborn as sns

from IPython.core.display  import HTML
from matplotlib            import pyplot as plt

## 0.1. Helper Functions

In [2]:
def jupyter_settings():
    %matplotlib inline
        
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = (20,8)
    plt.rcParams['font.size'] = 25
    
    display( HTML( '<style>.container { width:90% !important; }</style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )
    pd.set_option('display.float_format', lambda x: '%.3f' % x)
    
    sns.set()
    sns.set_theme(style = 'darkgrid', font_scale = 2)
    sns.set_style("darkgrid")
    sns.set(rc={'figure.figsize':(20,8)})

jupyter_settings()
    
def numerical_attributes(data):
    num_attributes = data.select_dtypes(include=['int64', 'float64'])
    
    # Central tendency = mean, median
    ct1 = pd.DataFrame(num_attributes.apply( np.mean) )
    ct2 = pd.DataFrame(num_attributes.apply( np.median) )

    # Dispersion = std, min, max, range, skew, kurtosis
    d1 = pd.DataFrame( num_attributes.apply( np.std ) )
    d2 = pd.DataFrame( num_attributes.apply( min ) )
    d3 = pd.DataFrame( num_attributes.apply( max ) )
    d4 = pd.DataFrame( num_attributes.apply( lambda x: x.max() - x.min() ) )
    d5 = pd.DataFrame( num_attributes.apply( lambda x: x.skew() ) )
    d6 = pd.DataFrame( num_attributes.apply( lambda x: x.kurtosis() ) )

    # Concatenate
    m = pd.concat( [d2, d3, ct1, ct2, d1, d5, d6], axis=1 ).reset_index()
    m.columns = ['attributes', 'min', 'max', 'mean', 'median', 'std', 'skew', 'kurtosis']

    return m

## 0.2. Loading Data

In [3]:
df_raw = pd.read_csv('../datasets/transformed_data.csv', low_memory=False)

In [4]:
df1 = df_raw.copy()

In [5]:
df1.head().T

,0,1,2,3,4
id,7129300520,6414100192,5631500400,2487200875,1954400510
date,2014-10-13,2014-12-09,2015-02-25,2014-12-09,2015-02-18
price,221900.000,538000.000,180000.000,604000.000,510000.000
bedrooms,3,3,2,4,3
bathrooms,1.000,2.250,1.000,3.000,2.000
sqft_living,1180,2570,770,1960,1680
sqft_lot,5650,7242,10000,5000,8080
floors,1.000,2.000,1.000,1.000,1.000
waterfront,0,0,0,0,0
view,0,0,0,0,0


## 2.0. FEATURE ENGINEERING

In [26]:
# Creating a column 'price_median'
aux1 = df1[['zipcode','price']].groupby('zipcode').median().reset_index()
aux1.columns = ['zipcode','price_median']
df2 = pd.merge(df1, aux1, on='zipcode', how='inner')

In [29]:
# Creating a column 'size' according to living room size in squarefeet
df2['size'] = df2['sqft_living'].apply(lambda x: 'tiny' if x < 1427 else
                                               'small' if (x > 1427) & (x < 1910) else
                                               'medium' if (x > 1910) & (x < 2550) else 'large')

In [35]:
# Creating a column 'old' if the property was built before 1965
df2['old'] = df2['yr_built'].apply(lambda x: 'yes' if x < 1965 else 'no')

In [46]:
# Creating a column 'buy' to check if the house is able to buy or not
for i in range (len(df2)):
    if df2.loc[(df2['price'] < df2['price_median']) & (df2['condition'] >= 2)]:
        df2['buy?'] = 'yes'
    else:
        df2['buy?'] = 'no'

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

## 2.1. Hypothesis List

**1.** Houses with no waterfront view are 30% cheaper on average.

**2.** Houses built before 1965 are 50% cheaper on average.

**3.** Houses without basement have lot size 40% bigger than houses with basement.

**4.** Related to the houses year of built, there is a price increase of 10% YoY.

**5.** Related to the date, houses with 3 bathrooms have a price increase of 15% MoM.

**6.** Houses which has more than 5 bedrooms are 25% more expensive on average, comparing by the same size.

**7.** Houses built before 1965 which were renovated are 60% more expensive than the houses not renovated from this period of time.

**8.** Houses on zipcode that has the biggest median of prices are responsible for 70% of profits.

**9.** Houses on zipcode that has the biggest median of prices correspond to 5% of the houses on sale.

**10.** Houses renovated in the last 10 years are 50% more valuable than the ones which had renovated 10-20 years before 2015.


## ## Saving Dataset

In [ ]:
df1.to_csv( path_or_buf= '../datasets/transformed_data.csv', index=False )